**Model Source:**<br />
https://github.com/kuangliu/pytorch-cifar/blob/49b7aa97b0c12fe0d4054e670403a16b6b834ddd/models/dla_simple.py

# Setup Variables

MNIST, FashionMNIST, GTSRB, Cifar10

In [1]:
import os
from pathlib import Path

base = Path().cwd()

if base.name != 'runtime-monitoring':
    os.chdir('../')
    base = Path().cwd()

base

PosixPath('/home/ah19/runtime-monitoring')

In [2]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

# Libraries

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from fastprogress import progress_bar
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

In [4]:
from utilities.utils import load_json, load_pickle, get_dataset, get_models
from utilities.pathManager import fetchPaths
from utilities.pcaFunctions import applyPCASingle

## Load model and settings

In [5]:
from models.mnist_model import MNIST_Model
from models.fashionmnist_model import FashionMNIST_CNN
from models.gtsrb_model import GTSRB_CNN
from models.cifar10_dla import Cifar10_DLA
from models.cifar10_model import Cifar10_CNN

from models.transformers import transformers

models = {
    'mnist': MNIST_Model,
    'fashionmnist': FashionMNIST_CNN,
    'gtsrb': GTSRB_CNN,
#     'cifar10': Cifar10_DLA,
    'cifar10': Cifar10_CNN
}


targets_datasets = {
    'MNIST': 'FashionMNIST',
    'FashionMNIST': 'MNIST',
    'GTSRB': 'Cifar10',
}

# Paths

In [6]:
def get_configs(DATASET):
    paths = fetchPaths(Path().cwd(), DATASET, '', False)

    configs = load_json(paths['configuration'])
    config = configs['configuration']
    optim = list(configs['configuration']['optimizer'].keys())[0]
    batch_size = configs['model_config']['batch_size']

    return config["dataset"], config["lhl_neurons"], config["flavors"], optim, batch_size

In [7]:
num_samples = 1_000


for base, target in targets_datasets.items():
    DATASET, lhl_neurons, flavors, optim, batch_size = get_configs(base.lower())
    
    TARGET_DATASET = targets_datasets[DATASET]
    
    for FLAVOR in flavors:
        for N in lhl_neurons:
            
            POSTFIX = f"{optim}-{batch_size}-{N}"
            FILENAME_POSTFIX = f"{DATASET}_{POSTFIX}"
            
            # generate paths
            base = Path().cwd()
            paths = fetchPaths(base, DATASET, POSTFIX)
            path_lhl = paths['lhl']
            path_lhl_data = paths['lhl_' + FLAVOR]
            path_bdd = paths['bdd'] / FLAVOR
            path_bdd.mkdir(exist_ok=True)
            path_target_data = paths['data'].parent / TARGET_DATASET
            
            # load pca and scaler objects
            if FLAVOR == 'scaler_pca':
                # load sacler object
                sacler_ = load_pickle(path_lhl / f'scaler.pkl')
                # load pca object
                pca_ = load_pickle(path_lhl / f'scaler_pca.pkl')
                
                
            # load config file
            configs = load_json(paths['configuration'])
            # model config
            model_setup = configs['model_setup']
            # saved model
            path_model = next(paths['saved_models'].glob('*.pth.tar'))

            # load trained model
            model_ = models[DATASET.lower()]
            transformer = transformers[DATASET.lower()]
            
            # load test data
            test_data = get_dataset(TARGET_DATASET, path_target_data, train=False)

            _, test_sample = train_test_split(
                torch.range(0, len(test_data.targets)-1, dtype=torch.int32),
                test_size=num_samples, shuffle=True, stratify=test_data.targets
            )
            # normalize data
            x_target = transformer['test'].transforms[1](torch.stack([test_data[i][0] for i in test_sample]))


            # torch 2.0 compile and parallel data training
            model_setup['last_hidden_neurons'] = N
            model = model_(**model_setup)
            model = torch.compile(model)

            # load model weights
            model.load_state_dict(torch.load(path_model)['model'])

            # eavluation mode
            model.eval()

            # get last hidden layer logits for target data
            logits_target, _ = model.output_last_layer(x_target)
            logits_target = logits_target.numpy()
            logits_target = pd.DataFrame(logits_target, columns=[f'x{i}' for i in range(logits_target.shape[1])])

            # project data to pca
            if FLAVOR == 'scaler_pca':
                # transform data
                logits_target = applyPCASingle(logits_target, sacler_, pca_, N)

            # target data
            df_logits_copy = logits_target.copy()
            df_logits_copy['y'] = 1
            df_logits_copy['true'] = 1
            df_logits_copy.to_csv(path_lhl_data / f"{FILENAME_POSTFIX}_{FLAVOR}_evaluation.csv", index=False)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
